# 第 5 讲：数值线性代数

In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

## 线性最小二乘

假如使用数据点 $y_i - \hat{y}$ 距离的平方和作为度量，即

$ E_2(a_0, a_1) = \sum_{i=0}^n (y_i - \hat{y})^2 = \sum_{i=0}^n[y_i - (a_0x_i + a_1)]^2$

这就是最小二乘(least squares). 它是求解最佳线性逼近的最方便的方法. 此外， 这个方法还包含误差的统计分布，对权重在数据点的分配策略比极小极大方法、绝对偏差方法更具理论优势.

分别对令其对 a0 和 a1 求偏导数

$$ \frac{\partial{E}}{\partial{a_0}} =0, \frac{\partial{E}}{\partial{a_1}} =0 $$

易得

$$ a_0m + a_1\sum_{i=1}^mx_i=\sum_{i=1}^my_i , a_0\sum_{i=1}^mx_i + a_1\sum_{i=1}^mx_i^2 = \sum_{i=1}^mx_iy_i$$

给出以下数据来验证Hooke定律的实验数据点，求拟合这些数据点的直线方程

In [2]:
xs = [1, 2 ,3, 4, 5, 6, 7, 8, 9, 10]
ys = [1.3, 3.5, 4.2, 5.0, 7.0, 8.8, 10.1, 12.5, 13.0, 15.6]

In [3]:
def linear_least_squares(xs, ys):
    n = len(xs)
    sum_x = sum(xs)
    sum_y = sum(ys)
    sum_x_2 = sum( x** 2 for x in xs )
    sum_xy = sum(xs[i] * ys[i] for i in range(n))
    a0 = (sum_x_2*sum_y - sum_xy*sum_x) / ( n * sum_x_2 - sum_x**2 )
    a1 = (n * sum_xy - sum_x * sum_y) / ( n * sum_x_2 - sum_x**2 )
    return a0, a1

In [4]:
a0, a1 = linear_least_squares(xs, ys)
print(a0)
print(a1)

-0.3600000000000044
1.5381818181818192


## Gram-Schmidt正交化过程

### 函数的正交

设$ f(x), g(x) \in C[a, b] $ $ (f, g) = \int_a^bw(x)f(x)g(x)dx = 0 $，若

$$ (f, g) = \int_a^bw(x)f(x)g(x) = 0 $$

则称$ f(x) $ 与 $ g(x) $在$ [a, b] $上带权$ w(x) $正交

### 正交函数族

当$ w(x) \equiv 1 $且 $ \phi_k(x) = x^k $时，推出正规方程

$$ \sum_{k=0}^na_k)\int_a^bw(x)\phi_k(x)\phi_j(x)dx = \int_a^bw(x)\phi_j(x)f(x)dx$$

设函数$ \phi_0, \phi_1, \ldots,\phi_k \in C[a,b]$，$ w(x) $是在$ [a,b] $上的权函数若

$$ (\phi_k, \phi_j) = \int_a^bw(x)\phi_k(x)\phi_j(x)dx$$

$$ (\phi_k, \phi_j) = \int_a^bw(x)\phi_k(x)\phi_j(x)dx=\begin{cases}
0 & j\neq k \\
A_j\neq 0 & j =k \\
\end{cases}$$

且当 $ k >= 2 $时，$ \phi_k(x) = (x - B_k)\phi_{k-1}(x) -  C_k\phi_{k-2}(x)$

其中$ \phi_0 = 1 $，$ \phi_1 =  x $，$ B_k =  \frac {x\phi_{k-1} \quad \phi_{k-1}} {\phi_{k-1} \quad \phi_{k-1}} $，$ C_k =  \frac {x\phi_{k-1} \quad \phi_{k-2}} {\phi_{k-2} \quad \phi_{k-2}} $ 

In [5]:
def Gram_Schmidt_Legendre(n):
    x = sp.symbols('x')
    B = [0, 0]
    phi = [1, x]
    C = [0, 0]

    for k in range(2, n+1):
        B.append( sp.integrate(x*phi[k-1]**2, (x, -1, 1)) / sp.integrate(phi[k-1]**2, (x, -1, 1)) )
        C.append( sp.integrate(x*phi[k-2]*phi[k-1], (x, -1, 1)) / sp.integrate(phi[k-2]**2, (x, -1, 1)) )
        phi.append( (x - B[k]) * phi[k-1] - C[k]*phi[k-2] )
        
    return phi[n]

In [6]:
sp.simplify(Gram_Schmidt_Legendre(3))

x*(x**2 - 3/5)

In [7]:
sp.simplify(Gram_Schmidt_Legendre(4))

x**4 - 6*x**2/7 + 3/35

In [8]:
sp.simplify(Gram_Schmidt_Legendre(5))

x*(63*x**4 - 70*x**2 + 15)/63

## 线性方程组

### 直接法

#### Gauss消去法（列主元法）

In [9]:
def Gaussian_Elimination(A, b):
    n = len(A)

    Ab = np.concatenate((A, b.reshape(n, 1)), axis=1)

    for i in range(n):
        max_row = i
        for j in range(i+1, n):
            if abs(Ab[j, i]) > abs(Ab[max_row, i]):
                max_row = j

        Ab[[i, max_row]] = Ab[[max_row, i]]

        for j in range(i+1, n):
            Ab[j] = Ab[j] - Ab[i] * (Ab[j][i] / Ab[i][i])

    x = np.zeros(n)
    for i in range(n-1, -1,  -1):
        x[i] = (Ab[i][n] - np.dot(Ab[i][i+1:n], x[i+1:n])) / Ab[i][i]

    return x

In [10]:
A = np.array([[2, 1, -1], [4, -6, 0], [-2, 7, 2]])
b = np.array([8, -2, 7])
Gaussian_Elimination(A,b)

array([ 2.5,  2. , -1. ])

In [11]:
x = np.linalg.solve(A, b)
x

array([ 2.5,  2. , -1. ])

### 迭代法

直接法算法运算量为O($ n^3 $)，随着矩阵规模的增长，运算量也随之**快速增长**

而对于大规模的稀疏矩阵，当前的首选方法是**迭代法**

三种经典迭代方法Jacobi，Gauss-Seidela，SOR

### Jacobi

In [12]:
def Jacobi_iteration(A, b, n):
    X = np.zeros(len(A)).reshape(-1,1)
    k = 1
    while k <= n:
        for i in range(len(A)):
            temp = 0
            for j in range(len(A)):
                if j != i:
                    temp += A[i][j]*X[j]
            X[i] = (b[i] - temp)/A[i][i]
        k = k + 1
    # print(X)
    return X

In [13]:
A = np.array([[10, -2, -1],[-2, 10, -1],[-1, -2, 5]])
b = np.array([3, 15, 10])
print(Jacobi_iteration(A, b, 0))
print(Jacobi_iteration(A, b, 1))
print(Jacobi_iteration(A, b, 2))
print(Jacobi_iteration(A, b, 3))
print(Jacobi_iteration(A, b, 4))

[[0.]
 [0.]
 [0.]]
[[0.3  ]
 [1.56 ]
 [2.684]]
[[0.8804  ]
 [1.94448 ]
 [2.953872]]
[[0.9842832 ]
 [1.99224384]
 [2.99375418]]
[[0.99782419]
 [1.99894025]
 [2.99914094]]


### SOR

当松弛参数为1时，即**Gauss-Seidela**

当松弛参数w < 1时，称为**低松弛方法**

当松弛参数w > 1时，称为**超松弛方法**

一般情况下取w > 1会获得较好的收敛情况

确定 SOR 的最优松弛因子是一件非常困难的事!

In [14]:
def Gauss_Seidela(A, b, w, n):
    X_new = np.zeros(len(A)).reshape(-1, 1)
    X_old = np.zeros(len(A)).reshape(-1, 1)
    k = 1
    while k <= n:
        for i in range(len(A)):
            X_new[i] = (1 - w) * X_old[i] + (w / A[i][i]) * (b[i] - sum([A[i][j] * X_new[j] for j in range(i)])
                                                           - sum([A[i][j] * X_old[j] for j in range(i+1, len(A))]))
            X_old[i] = X_new[i]
        k = k+1
    return X_new

In [15]:
Gauss_Seidela(A, b, 1, 10)

array([[0.99999999],
       [1.99999999],
       [2.99999999]])

## 基于 Householder 变换 

In [28]:
def householder_reflection(A):
    """Householder变换"""
    (r, c) = np.shape(A)
    Q = np.identity(r)
    R = np.copy(A)
    for cnt in range(r - 1):
        x = R[cnt:, cnt]
        e = np.zeros_like(x)
        e[0] = np.linalg.norm(x)
        u = x - e
        v = u / np.linalg.norm(u)
        Q_cnt = np.identity(r)
        Q_cnt[cnt:, cnt:] = Q_cnt[cnt:, cnt:] - 2.0 * np.outer(v, v)
        R = np.dot(Q_cnt, R)
        Q = np.dot(Q, Q_cnt)
    return (Q, R)

In [29]:
A = np.array([[4, 1, -2, 2], [1, 2, 0, 1], [-2, 0, 3, -2], [2, 1, -2, 1]])
x1, x2 = householder_reflection(A)
print(x1)
print(x2)
np.dot(x1, x2)

[[ 0.8        -0.15096588  0.57749944  0.06085806]
 [ 0.2         0.90579529 -0.07874992  0.36514837]
 [-0.4         0.34506487  0.69562432 -0.4868645 ]
 [ 0.4         0.19409899 -0.41999959 -0.79115481]]
[[ 5.00000000e+00  1.60000000e+00 -3.60000000e+00  3.00000000e+00]
 [ 0.00000000e+00  1.85472370e+00  9.48928404e-01  1.07832773e-01]
 [ 0.00000000e+00  2.49536316e-16  1.77187327e+00 -7.34999282e-01]
 [ 0.00000000e+00  1.91837913e-16 -1.32693068e-16  6.69438681e-01]]


array([[ 4.00000000e+00,  1.00000000e+00, -2.00000000e+00,
         2.00000000e+00],
       [ 1.00000000e+00,  2.00000000e+00, -3.60140743e-16,
         1.00000000e+00],
       [-2.00000000e+00, -2.35286067e-16,  3.00000000e+00,
        -2.00000000e+00],
       [ 2.00000000e+00,  1.00000000e+00, -2.00000000e+00,
         1.00000000e+00]])